# Import
---


In [0]:
import time
import pandas as pd

import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets   as datasets

# Model
---
AlexNetを基にした小さい画像用CNNモデル

In [0]:
class CNN(nn.Module):
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(  1,  32, kernel_size=5, padding=2), nn.ReLU(inplace=True), nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d( 32,  64, kernel_size=5, padding=2), nn.ReLU(inplace=True), nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d( 64, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(), nn.Linear(256*6*6, 4096), nn.ReLU(inplace=True),
            nn.Dropout(), nn.Linear(   4096, 4096), nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(-1, 256*6*6) # flatten
        x = self.classifier(x)
        return x

In [0]:
if torch.cuda.is_available(): device = 'cuda'
else                        : device = 'cpu' 

net = CNN(num_classes=10).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

# Load Dataset
---

In [4]:
toTensor  = transforms.ToTensor()
normalize = transforms.Normalize((0.5, ), (0.5, ))
transform = transforms.Compose([toTensor, normalize])

train_set = datasets.FashionMNIST(root='../data', download=True, train=True , transform=transform)
valid_set = datasets.FashionMNIST(root='../data', download=True, train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=1000, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=1000, shuffle=True)

print('size  :', train_set[0][0].size())
print('train :', len(train_set))
print('valid :', len(valid_set))

size  : torch.Size([1, 28, 28])
train : 60000
valid : 10000


# Training & Validation
---

In [0]:
def train():
    net.train()

    # mini-batch learning
    epoch_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        # forward
        t = net(x)
        # cal loss
        loss = loss_func(t, y)
        epoch_loss += loss.item()
        # backward
        loss.backward()
        # update model
        optimizer.step()
        optimizer.zero_grad()

    return epoch_loss

In [0]:
def valid():
    net.eval()

    with torch.no_grad():
        # mini-batch validation (for save memory)
        epoch_accu = 0
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            # forward
            t = net(x)
            # cal accuracy
            _, p = torch.max(t.data, dim=1)
            accu = (p == y).sum()
            epoch_accu += accu.item()

    return epoch_accu / len(valid_set)

In [7]:
max_epoch = 30

results = []
for epoch in range(max_epoch):
    st = time.time()
    loss = train()
    md = time.time()
    accu = valid()
    ed = time.time()

    results.append([epoch+1, md-st, loss, ed-md, accu])
    print(f'{epoch+1:>2} | {md - st:5.2f}s | {loss:>6.3f} | {ed - md:5.2f}s | {accu:.2%}')

 1 | 11.48s | 69.406 |  1.61s | 76.92%
 2 | 11.71s | 30.113 |  1.64s | 83.25%
 3 | 12.03s | 23.493 |  1.61s | 86.04%
 4 | 11.85s | 20.114 |  1.63s | 87.73%
 5 | 11.64s | 17.256 |  1.62s | 88.57%
 6 | 11.49s | 16.181 |  1.57s | 89.58%
 7 | 11.49s | 14.556 |  1.59s | 90.08%
 8 | 11.61s | 13.097 |  1.63s | 90.63%
 9 | 11.61s | 12.199 |  1.64s | 90.55%
10 | 11.70s | 11.681 |  1.64s | 90.81%
11 | 11.72s | 10.871 |  1.65s | 90.67%
12 | 11.62s |  9.871 |  1.65s | 91.54%
13 | 11.61s |  9.460 |  1.60s | 91.44%
14 | 11.63s |  8.607 |  1.62s | 91.13%
15 | 11.62s |  8.089 |  1.64s | 90.93%
16 | 11.61s |  7.564 |  1.61s | 91.39%
17 | 11.62s |  6.838 |  1.58s | 91.10%
18 | 11.61s |  6.249 |  1.60s | 91.26%
19 | 11.61s |  5.683 |  1.61s | 91.42%
20 | 11.62s |  5.021 |  1.61s | 90.83%
21 | 12.14s |  5.145 |  1.58s | 91.17%
22 | 11.63s |  4.193 |  1.59s | 91.21%
23 | 11.63s |  4.354 |  1.61s | 91.52%
24 | 11.62s |  3.620 |  1.59s | 91.65%
25 | 11.61s |  3.418 |  1.61s | 91.26%
26 | 11.62s |  2.844 |  1

# Save Model & Result
---

In [0]:
torch.save(net.state_dict(), 'models/FMNIST.pt')

In [0]:
columns = ['epoch', 'train time(s)', 'loss', 'valid time(s)', 'accu']
df = pd.DataFrame(data=results, columns=columns).set_index('epoch')
df.to_csv('./results/FMNIST.csv')

# Save Targets & Predicts
---

In [0]:
net.load_state_dict(torch.load('models/FMNIST.pt'))

ps, ys = [], []
for x, y in valid_loader:
    x, y = x.to(device), y.to(device)

    t = net(x)
    _, p = torch.max(t.data, dim=1)

    ps.append(p.cpu())
    ys.append(y.cpu())

ps = torch.cat(ps).data.detach().numpy()
ys = torch.cat(ys).data.detach().numpy()

labels = datasets.FashionMNIST.classes
ps = [labels[p] for p in ps]
ys = [labels[y] for y in ys]

In [0]:
df1 = pd.DataFrame(data={'target' :ys})
df2 = pd.DataFrame(data={'predict':ps})

df1.to_csv('outputs/target_FMNIST.csv')
df2.to_csv('outputs/predict_FMNIST.csv')